### <center>курс GeekBrains "Методы сбора и обработки данных из сети Интернет"<center>

### <center>Домашнее задание к уроку №5 "Selenium в Python"<center>

**Вариант I** <br>
Написать программу, которая собирает входящие письма из своего или тестового почтового ящика
и сложить данные о письмах в базу данных (от кого, дата отправки, тема письма, текст письма полный)
Логин тестового ящика: study.ai_172@mail.ru
Пароль тестового ящика: NextPassword172#

**Подключение библиотек и скриптов**

In [1]:
import time
from pprint import pprint
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
from hashlib import md5

**Функция формирующая hash-строку из полученного словаря dict**

In [2]:
# Функция формирующая hash-строку из полученного словаря dict
def id_hash(dict):
    temp = str(dict).encode('utf-8')
    result = md5(temp).hexdigest()

    return result

**Функция записывающая данные из полученного списка словарей в mongo базу данных base**

In [3]:
def mongo_dict_insert(list, base):
    '''настоящая функция записывает данные из полученного списка словарей list в базу данных base
       важный момент: в составе словаря должен быть _id по которому настоящая функция определяет будет ли сделана запись,
       если запись с таким _id уже есть в base, то будет поднято exception и запись не произойдет
    :param list: список словарей
    :return [add_num, except_num]: сколько словарей [добавлено, уже есть в базе данных]
    '''
    
    add_num = 0
    except_num = 0
    
    for n in list:
        try:                           # перед записью в mongo происходит проверка на уникальность _id
            base.insert_one(n)         
            add_num += 1
        except DuplicateKeyError:      # если запись с таким _id уже есть, то поднимается exception
            except_num += 1
            #print(f"Document with id = {n['_id']} already exists")

    print(f'на входе имеем список содержащий {len(list)} словарей')
    print(f'{add_num} словарей было добавлено в базу данных {base}')
    print(f'{except_num} словарей из предложенного списка уже есть в базе данных {base}')
            
    return [add_num, except_num]

**поехали**

In [4]:
options = Options()
options.add_argument("start-maximized")   # браузер на весь экран

s = Service('./chromedriver')
driver = webdriver.Chrome(service=s, options=options)

**для задержки следующих действий прежде чем не загрузятся все элементы текущей страницы**

In [5]:
driver.implicitly_wait(15)

**получаем доступ к странице "Входящие"**

In [6]:
driver.get('https://account.mail.ru/login')

elem = driver.find_element(By.XPATH, '//body/div[@id="login-content"]//input[contains(@name, "username")]')
#elem = driver.find_element(By.NAME, 'username')
elem.send_keys("study.ai_172@mail.ru")
elem.send_keys(Keys.ENTER)

time.sleep(1) # вроде и без нее работает, но оставляю на всякий случай

elem = driver.find_element(By.XPATH, '//body/div[@id="login-content"]//input[contains(@name, "password")]')
elem.send_keys("NextPassword172#")
elem.send_keys(Keys.ENTER)

In [7]:
# тег в котором есть информация о количестве писем в папке Входящие
# <a class="nav__item js-shortcut nav__item_active nav__item_shortcut nav__item_child-level_0" href="/inbox/" title="Входящие, 670 писем, 16 непрочитанных">

In [8]:
# читаем всплывающую подсказку "Входящие, 670 писем, 16 непрочитанных"
elem = driver.find_element(By.XPATH, '//a[@href="/inbox/"]')
temp_str = elem.get_attribute('title')
temp_str[9:14]

' 691 '

In [9]:
# определяем количество страниц
n_mails = 20   # для отладки
# n_mails = int(temp_str[9:14])    # 670 - общее количество писем в папке "Входящие" согласно всплывающей подсказке
n_pages = int(n_mails / 10)      # n_pages = 67 - примерно по 10 писем на странице

**проходимся по каждой странице блока "Входящие" и собираем link каждого письма в множество set_of_links**

In [10]:
set_of_links = set()

for i in range(0, n_pages):
    
    elems = driver.find_elements(By.CLASS_NAME, 'llc')
    last = len(elems)
    
    for el in elems:
        link = el.get_attribute('href').split('?')
        set_of_links.add(link[0])

    actions = ActionChains(driver)
    #actions.key_down(Keys.PAGE_DOWN) # не получилось
    actions.move_to_element(elems[last - 1])
    actions.perform()

    time.sleep(1)

**формируем список добытых ссылок links**

In [11]:
links = list(set_of_links)

**собираем итоговый список словарей mail_list в котором будут ключевые данные по входящим письмам**

In [12]:
mail_list = []

for link in links:
    
    driver.get(link)

    time.sleep(1)
    
    subject = driver.find_element(By.CLASS_NAME, "thread-subject").text
    contact = driver.find_element(By.CLASS_NAME, "letter-contact").text
    date_ = driver.find_element(By.CLASS_NAME, "letter__date").text
    body = driver.find_element(By.CLASS_NAME, "letter__body").text
    body = body.replace('\n', '')
    body = body.replace('\t', '')
    
    mail = {}
    mail['subject'] = subject
    mail['from_'] = contact
    mail['date_'] = date_
    mail['_id'] = id_hash(mail)
    mail['body'] = body
    
    mail_list.append(mail)

In [13]:
len(mail_list)

32

**подключаем Mongo и работаем с базой данных 'news_db'**

In [14]:
client = MongoClient('127.0.0.1', 27017)
db = client['mails_db']
mails = db.mails
#news.delete_many({})    # включаем эту строку если надо очистить базу данных 

**инициируем работу функции mongo_dict_insert, которая записывает данные из news_mail_ru_list в 'mails_db'**

In [15]:
mongo_dict_insert(mail_list, mails)

на входе имеем список содержащий 32 словарей
26 словарей было добавлено в базу данных Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'mails_db'), 'mails')
6 словарей из предложенного списка уже есть в базе данных Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'mails_db'), 'mails')


[26, 6]

**Вариант II**<br>
Написать программу, которая собирает товары «В тренде» с сайта техники mvideo и складывает данные в БД. 
Сайт можно выбрать и свой. Главный критерий выбора: динамически загружаемые товары

In [17]:
# https://www.mvideo.ru/

**этот вариант не успевают довести "до ума" и "вычистить" все недоделки,
основной скелет вроде сделан**

In [5]:
options = Options()
options.add_argument("start-maximized")   # браузер на весь экран

s = Service('./chromedriver')
driver = webdriver.Chrome(service=s, options=options)

In [6]:
driver.get('https://www.mvideo.ru/')

In [7]:
#driver.execute_script("window.scrollTo(0,100)")
driver.execute_script("window.scrollTo(0,1500)")
time.sleep(1)

In [8]:
elem = driver.find_element(By.XPATH, "//button//span[contains(text(), 'В тренде')]")

In [9]:
elem.click()

In [10]:
#/html/body/mvid-root/div/mvid-primary-layout/mvid-layout/div/main/ng-component/div/mvid-shelf-group/mvid-carousel/div[1]/div/mvid-product-cards-group/div[2]
#//mvid-root//mvid-primary-layout/mvid-layout/div/main/ng-component/div/mvid-shelf-group/mvid-carousel//mvid-product-cards-group/div
# //mvid-root//mvid-primary-layout/mvid-layout//mvid-shelf-group/mvid-carousel//mvid-product-cards-group/div
_x_path = '//mvid-shelf-group/mvid-carousel//mvid-product-cards-group/div//a[@class="img-with-badge ng-star-inserted"]'

In [11]:
products = driver.find_elements(By.XPATH, _x_path)
len(products)

16

In [12]:
product_links = []
for product in products:    
    
    product_links.append(product.get_attribute('href'))
    
#print(product_links)

In [13]:
products_list = []

for link in product_links:    

    driver.get(link)
  
    time.sleep(3)
    
#//mvid-root/div/mvid-primary-layout/mvid-layout/div/main/mvid-pdp/mvid-pdp-general/div/mvid-general-details/div/div[1]/h1     
    product_name = driver.find_element(By.XPATH, './/h1').text
    
#     product_price = driver.find_element(By.XPATH, './/span[@class="price__main-value"]').text
    
    product = {}
    product['link'] = link
    product['name'] = product_name
#     product['price'] = product_price
    product['_id'] = id_hash(product)
    
    products_list.append(product)  

In [14]:
print(products_list)

[{'link': 'https://www.mvideo.ru/products/smartfon-apple-iphone-11-128gb-black-mhdh3ru-a-30052942', 'name': 'Смартфон Apple iPhone 11 128GB Black (MHDH3RU/A)', '_id': 'ec98216aa201165c06492b2558811751'}, {'link': 'https://www.mvideo.ru/products/naushniki-apple-airpods-3-go-pokoleniya-mme73ru-a-50163928', 'name': 'Наушники Apple AirPods 3-го поколения (MME73RU/A)', '_id': 'd34c7b9d258dc415ba6ed53c847e375c'}, {'link': 'https://www.mvideo.ru/products/smartfon-xiaomi-redmi-note-11-nfc-4gb128gb-twilight-blue-30062384', 'name': 'Смартфон Xiaomi Redmi Note 11 NFC 4GB+128GB Twilight Blue', '_id': 'aea1a58fd434f7497e6d86febd8fbeb1'}, {'link': 'https://www.mvideo.ru/products/televizor-vityaz-24lh0201-10021226', 'name': 'Телевизор Витязь 24LH0201', '_id': '5dd2324aa0180485cdd82d2e4164edb8'}, {'link': 'https://www.mvideo.ru/products/elektricheskii-samokat-kugoo-kirin-s1-pro-kgo-kgs1pro-bk-10025947', 'name': 'Электрический самокат Kugoo Kirin S1 Pro (KGO-KGS1PRO-BK)', '_id': '929bc906ad8eac2fb6d7db

In [15]:
client = MongoClient('127.0.0.1', 27017)
db = client['products_db']
products = db.products
#news.delete_many({})    # включаем эту строку если надо очистить базу данных 

In [16]:
mongo_dict_insert(products_list, products)

на входе имеем список содержащий 16 словарей
0 словарей было добавлено в базу данных Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'products_db'), 'products')
16 словарей из предложенного списка уже есть в базе данных Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'products_db'), 'products')


[0, 16]